# Dataset and Dataloader

面向对象？

面向对象：
- class：类，一个比较抽象的东西，它可以代表一部分拥有共同属性(attribute)和共同行为(action)的事物。
  Person:name,sex,age....(属性);walk,sleep,jump(行为)；
  
  给类的每一个属性赋值，那么这个过程其实就是：**实例化**

  实例化：类 -> 对象
- object：对象，一个比较具体的东西。（instance）

面向对象的几大特征：继承、多态。。。

Dataset运用**继承**这个特征。

继承？

Person{}

YellowPeople{}

person有的属性和行为，黄种人一定有？肯定的。

继承：黄种人子类继承了person这个父类。


Dataset：父类，是所有开发人员训练、测试使用的所有数据集的一个模板或者是抽象。

dogvscat-dataset：子类，是一个具体的是数据集，它继承Dataset这个父类的所有方法(行为,function)和属性。

## 如何去重写Dataset，定义自己的数据集

In [35]:
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd # 用这个进行csv的生成
import os
import numpy as np
from PIL import Image
from torchvision import transforms

In [2]:
help(Dataset)

Help on class Dataset in module torch.utils.data.dataset:

class Dataset(typing.Generic)
 |  Dataset(*args, **kwds)
 |  
 |  An abstract class representing a :class:`Dataset`.
 |  
 |  All datasets that represent a map from keys to data samples should subclass
 |  it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
 |  data sample for a given key. Subclasses could also optionally overwrite
 |  :meth:`__len__`, which is expected to return the size of the dataset by many
 |  :class:`~torch.utils.data.Sampler` implementations and the default options
 |  of :class:`~torch.utils.data.DataLoader`.
 |  
 |  .. note::
 |    :class:`~torch.utils.data.DataLoader` by default constructs a index
 |    sampler that yields integral indices.  To make it work with a map-style
 |    dataset with non-integral indices/keys, a custom sampler must be provided.
 |  
 |  Method resolution order:
 |      Dataset
 |      typing.Generic
 |      builtins.object
 |  
 |  Methods defined 

getitem方法返回的是数据项并非数据集

(1)生成csv文件

In [8]:
dataset_dir = './dogvscat'
# ImageFolder
classes = os.listdir(dataset_dir)

info_array = [] # N,3的array
col = ['filename','filepath','label']
for kindname in os.listdir(dataset_dir):
    classpath = dataset_dir + '/' + kindname
    for filename in os.listdir(classpath):
        filepath = classpath + '/' + filename
        label = classes.index(kindname) # str->index int
        info_array.append([filename,filepath,label])
info_array = np.array(info_array)
df = pd.DataFrame(info_array, columns=col)
df.to_csv('./dogvscat_dataset.csv',encoding='utf-8')

(2)重写Dataset

In [36]:
class Dogvscat_dataset(Dataset):
    def __init__(self,dataset_dir,csv_path, resize_shape):
       # init方法一般要编写数据的transformer、数据的基本参数。
       self.dataset_dir = dataset_dir
       self.csv_path =csv_path
       self.shape = resize_shape
       self.df = pd.read_csv(self.csv_path,encoding='utf-8')
       self.transformer = transforms.Compose([
           transforms.Resize(self.shape),
           transforms.ToTensor(),
       ])
    
    def __len__(self):  # 返回数据规模
        return len(self.df)
    
    def __getitem__(self,idx):  # getitem,idx=index
        x_train = self.transformer(Image.open(self.df['filepath'][idx]))
        y_train = self.df['label'][idx]
        
        return x_train,y_train


In [37]:
data_ds = Dogvscat_dataset('./dogvscat','./dogvscat_dataset.csv',(224,224))

len(data_ds)

8000

(3)生成dataloader

In [38]:
num_sample = len(data_ds)
train_percent = 0.8
train_num = int(train_percent*num_sample)
test_num = num_sample-train_num
train_ds, test_ds = random_split(data_ds, [train_num,test_num])

print(len(train_ds))
print(len(test_ds))

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=32, shuffle=True)

6400
1600


(4)迭代数据

In [41]:
for x_train,y_train in iter(train_dl):
    print(x_train)
    print(y_train)
    break

tensor([[[[0.1333, 0.2078, 0.3059,  ..., 0.3686, 0.2941, 0.3882],
          [0.1922, 0.2471, 0.2510,  ..., 0.3373, 0.3294, 0.4353],
          [0.2510, 0.3686, 0.5176,  ..., 0.3294, 0.4235, 0.3922],
          ...,
          [0.4706, 0.4275, 0.3961,  ..., 0.5137, 0.5490, 0.5137],
          [0.5804, 0.5098, 0.4314,  ..., 0.4392, 0.4353, 0.4549],
          [0.5255, 0.5412, 0.5216,  ..., 0.5725, 0.5137, 0.4196]],

         [[0.1804, 0.2431, 0.3333,  ..., 0.3373, 0.2471, 0.3412],
          [0.2039, 0.2549, 0.2510,  ..., 0.2588, 0.2353, 0.3412],
          [0.2157, 0.3333, 0.4706,  ..., 0.1961, 0.2627, 0.2235],
          ...,
          [0.3843, 0.3412, 0.3020,  ..., 0.4549, 0.4824, 0.4471],
          [0.5020, 0.4314, 0.3451,  ..., 0.3765, 0.3608, 0.3765],
          [0.4510, 0.4667, 0.4353,  ..., 0.5059, 0.4314, 0.3333]],

         [[0.0706, 0.1373, 0.2275,  ..., 0.2078, 0.1176, 0.2078],
          [0.1098, 0.1608, 0.1608,  ..., 0.1451, 0.1255, 0.2275],
          [0.1412, 0.2627, 0.4000,  ..., 0

dataset才是真正地编写成功了。